In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import pydae.svg_tools as svgt
from pydae import ssa
#%config InlineBackend.figure_format = 'svg'
import pandas as pd

In [2]:
%matplotlib widget

In [3]:
from bess_1 import bess_1_class

In [4]:
df_v = pd.read_excel(r'C:\Users\jmmau\ownCloud\EASY-RES PROJECT\WP2\T2.2\FENECON\Tests\Diagramms_Discharging Test\1DischargingTest_BatteryVoltage.xlsx',nrows=2438)
df_p = pd.read_excel(r'C:\Users\jmmau\ownCloud\EASY-RES PROJECT\WP2\T2.2\FENECON\Tests\Diagramms_Discharging Test\p_u.xlsx',usecols='A:C',nrows=2438)
df_v['time'] = np.linspace(0,203*60,len(df_v['Time']))
df_p['time'] = np.linspace(0,203*60,len(df_p['Time']))
df_v_2 = df_v.dropna()
df_p_2 = df_p.dropna() 

In [5]:
E_bess = 3000.0
V_u_max = 53.0
V_u_min = 52

E_Joules = E_bess*3600
C_u = 2*E_Joules/((V_u_max**2 - V_u_min**2))

# E_max = 0.5*C_ucap*(V_ucap_max*V_ucap_max - V_ucap_min*V_ucap_min)
# soc = 0.5*C_ucap*(e_u*e_u - V_ucap_min*V_ucap_min)/E_max
#V_ucap_0 = (2*soc_0*E_Joules/(0.5*C_ucap) + V_u_min**2)**0.5
C_u

205714.2857142857

In [6]:
b1 = bess_1_class()
b1.simulate([{'Dt':1e-1, 'K_p':0.001,'K_i':0.001,'R_stor':0.02,'p_u_ref':0,
              'soc_ref':1.0,'V_u_min':V_u_min,'V_u_max':V_u_max,'C_u':C_u},
            ],0);
b1.report_u()

p_u_ref =  0.00


In [7]:
b1 = bess_1_class()
b1.initialize([{'Dt':1e-1, 'K_p':0.001,'K_i':0.001,'R_stor':0.022,'p_u_ref':0,
              'soc_ref':1.0,'V_u_min':V_u_min,'V_u_max':V_u_max,'C_u':C_u},
            ],0);

t_prev = 0
for t in df_p_2['time'].values:
    p_u_ref = np.interp(t,df_p_2['time'].values,df_p_2['Value'].values)
    if t>t_prev:
        b1.run([{'t_end':t,'p_u_ref':p_u_ref}])
        t_prev = t
b1.post();

In [8]:
fig, axes = plt.subplots(nrows=3,ncols=1, figsize=(6, 6), dpi=80)

axes[0].plot(b1.T/60, b1.get_values('p_u_soc'), label=f'p_u_soc')
axes[0].plot(b1.T/60, b1.get_values('p_u'), label=f'p_u')
axes[1].plot(b1.T/60, b1.get_values('soc'), label=f'soc')
axes[2].plot(b1.T/60, b1.get_values('v_u'), label=f'v_u (model)')
axes[2].plot(df_v_2['time'].values/60,df_v_2['Real Value in Volt'].values,  label=f'v_u (FENECON)')
#axes.set_ylabel('$\\theta (º)$')
#axes[2].set_xlabel('Time (s)')
for ax in axes:
    ax.legend(ncol=2)
    ax.set_xlim((0,165))
axes[2].set_ylim((51.25,54.5))
axes[2].set_xlabel('Time (minutes)')

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …